# Game Recommender for Games on Steam

## Project Description

In this project we are going to create three different recommenders for games* on Steam.
1. Knowledge-based recommender (a filter).
2. Content-based recommender, based on app description.
3. Content-based recommender, based on tags and developers (app metadata).

We'll also use Streamlit to create the application for this recommenders.


**datasets also contain information about soundtracks, software and films, but the main focus of this recommenders is games.*

## Set Up
### Downloading Files
#### ***! VERY IMPORTANT***
***Please consider running code in the next code cell - this files are crucial for running recommender applications***.

You don't need to wait till the end of code execution and can proceed further in observing the notebook.

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UlXewxo8rSvL-hAqF8TM0BYnlB_judDW' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1UlXewxo8rSvL-hAqF8TM0BYnlB_judDW" -O recommender_files.zip && rm -rf /tmp/cookies.txt
!unzip recommender_files.zip &> /dev/null
!rm -rf __MACOSX & rm -rf recommender_files.zip

### Importing Libraries and Updating Settings

In [ ]:
import os
import json
import numpy as np
import pandas as pd


pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option("display.precision", 4)

## Getting the Data
### About the Data

We'll use data from two different sources:

1. Two datasets from [Game Recommendations on Steam Dataset](https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam/data?select=games.csv):
    - **games.csv** as the main source of information.
        - Contains information about game *title*, *release date*, *rating*, *price* and *compatibility with different platforms*.
    - **games_metadata.json** for dame metadata.
        - Contains information about game *tags* and *descriptions*.
2. One dataset from [Steam Games Dataset](https://www.kaggle.com/datasets/fronkongames/steam-games-dataset/data?select=games.json):
    - **games.json** for some additional information about games (*image*, *developers*).

### Loading Data into Data Frames

Dataset with main data.

In [ ]:
data = pd.read_csv("recommender_files/data/games.csv", index_col = "app_id")

data.head()

,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
app_id,,,,,,,,,,,,
13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


Dataset with metadata.

In [ ]:
# this file is not in a "valid json" format so we need to read it line by line
metadata = [json.loads(line)
        for line in open('recommender_files/data/games_metadata.json', 'r', encoding='utf-8')]

In [ ]:
meta = pd.DataFrame(metadata)
meta = meta.set_index("app_id")

meta.head()

,description,tags
app_id,,
13500,Enter the dark underworld of Prince of Persia ...,"[Action, Adventure, Parkour, Third Person, Gre..."
22364,,[Action]
113020,Monaco: What's Yours Is Mine is a single playe...,"[Co-op, Stealth, Indie, Heist, Local Co-Op, St..."
226560,Escape Dead Island is a Survival-Mystery adven...,"[Zombies, Adventure, Survival, Action, Third P..."
249050,Dungeon of the Endless is a Rogue-Like Dungeon...,"[Roguelike, Strategy, Tower Defense, Pixel Gra..."


Dataset with additional data.

In [ ]:
add_set = {}
with open('recommender_files/data/games.json', 'r', encoding='utf-8') as fin:
  text = fin.read()
  add_set = json.loads(text)

In [ ]:
additions = pd.DataFrame(add_set).T
additions.index = additions.index.astype(np.int64)

additions.head()

,name,release_date,required_age,price,dlc_count,detailed_description,about_the_game,short_description,reviews,header_image,website,support_url,support_email,windows,mac,linux,metacritic_score,metacritic_url,achievements,recommendations,notes,supported_languages,full_audio_languages,packages,developers,publishers,categories,genres,screenshots,movies,user_score,score_rank,positive,negative,estimated_owners,average_playtime_forever,average_playtime_2weeks,median_playtime_forever,median_playtime_2weeks,peak_ccu,tags
20200,Galactic Bowling,"Oct 21, 2008",0,19.99,0,Galactic Bowling is an exaggerated and stylize...,Galactic Bowling is an exaggerated and stylize...,Galactic Bowling is an exaggerated and stylize...,,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.galacticbowling.net,,,True,False,False,0,,30,0,,[English],[],"[{'title': 'Buy Galactic Bowling', 'descriptio...",[Perpetual FX Creative],[Perpetual FX Creative],"[Single-player, Multi-player, Steam Achievemen...","[Casual, Indie, Sports]",[https://cdn.akamai.steamstatic.com/steam/apps...,[http://cdn.akamai.steamstatic.com/steam/apps/...,0,,6,11,0 - 20000,0,0,0,0,0,"{'Indie': 22, 'Casual': 21, 'Sports': 21, 'Bow..."
655370,Train Bandit,"Oct 12, 2017",0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,THE LAW!! Looks to be a showdown atop a train....,THE LAW!! Looks to be a showdown atop a train....,,https://cdn.akamai.steamstatic.com/steam/apps/...,http://trainbandit.com,,support@rustymoyher.com,True,True,False,0,,12,0,,"[English, French, Italian, German, Spanish - S...",[],"[{'title': 'Buy Train Bandit', 'description': ...",[Rusty Moyher],[Wild Rooster],"[Single-player, Steam Achievements, Full contr...","[Action, Indie]",[https://cdn.akamai.steamstatic.com/steam/apps...,[http://cdn.akamai.steamstatic.com/steam/apps/...,0,,53,5,0 - 20000,0,0,0,0,0,"{'Indie': 109, 'Action': 103, 'Pixel Graphics'..."
1732930,Jolt Project,"Nov 17, 2021",0,4.99,0,Jolt Project: The army now has a new robotics ...,Jolt Project: The army now has a new robotics ...,"Shoot vehicles, blow enemies with a special at...",,https://cdn.akamai.steamstatic.com/steam/apps/...,,,ramoncampiaof31@gmail.com,True,False,False,0,,0,0,,"[English, Portuguese - Brazil]",[],"[{'title': 'Buy Jolt Project', 'description': ...",[Campião Games],[Campião Games],[Single-player],"[Action, Adventure, Indie, Strategy]",[https://cdn.akamai.steamstatic.com/steam/apps...,[http://cdn.akamai.steamstatic.com/steam/apps/...,0,,0,0,0 - 20000,0,0,0,0,0,[]
1355720,Henosis™,"Jul 23, 2020",0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,HENOSIS™ is a mysterious 2D Platform Puzzler w...,HENOSIS™ is a mysterious 2D Platform Puzzler w...,,https://cdn.akamai.steamstatic.com/steam/apps/...,https://henosisgame.com/,https://henosisgame.com/,info@henosisgame.com,True,True,True,0,,0,0,,"[English, French, Italian, German, Spanish - S...",[],"[{'title': 'Buy Henosis™', 'description': '', ...",[Odd Critter Games],[Odd Critter Games],"[Single-player, Full controller support]","[Adventure, Casual, Indie]",[https://cdn.akamai.steamstatic.com/steam/apps...,[http://cdn.akamai.steamstatic.com/steam/apps/...,0,,3,0,0 - 20000,0,0,0,0,0,"{'2D Platformer': 161, 'Atmospheric': 154, 'Su..."
1139950,Two Weeks in Painland,"Feb 3, 2020",0,0.0,0,ABOUT THE GAME Play as a hacker who has arrang...,ABOUT THE GAME Play as a hacker who has arrang...,Two Weeks in Painland is a story-driven game a...,,https://cdn.akamai.steamstatic.com/steam/apps/...,https://www.unusual-games.com/home/,https://www.unusual-games.com/contact/,welistentoyou@unusual-games.com,True,True,False,0,,17,0,This Game may contain content not appropriate ...,"[English, Spanish - Spain]",[],[],[Unusual Games],[Unusual Games],"[Single-player, Steam Achievements]","[Adventure, Indie]",[https://cdn.akamai.steamstatic.com/steam/apps...,[http://cdn.akamai.steamstatic.com/steam/apps/...,0,,50,8,0 - 20000,0,0,0,0,0,"{'Indie': 42, 'Adventure': 41, 'Nudity': 22, '..."


Fetching `images` column beforehand to join with our main dataset later.

In [ ]:
images = additions[["header_image"]].copy()

images.head()

,header_image
20200,https://cdn.akamai.steamstatic.com/steam/apps/...
655370,https://cdn.akamai.steamstatic.com/steam/apps/...
1732930,https://cdn.akamai.steamstatic.com/steam/apps/...
1355720,https://cdn.akamai.steamstatic.com/steam/apps/...
1139950,https://cdn.akamai.steamstatic.com/steam/apps/...


### Cleaning and Preparing the Data

Joining main dataset with metadata.

In [ ]:
df = data.join(meta)

Cleaning rows with no information about tags.

In [ ]:
empty_tags_ids = list()
for index, value in zip(df["tags"].index, df["tags"]):
    if value == list():
        empty_tags_ids.append(index)

In [ ]:
df = df.drop(index = empty_tags_ids)

Dropping columns without description.

In [ ]:
empty_description_ids = df.loc[df["description"] == ""].index

df = df.drop(index = empty_description_ids)

Joining cleaned dataset with image urls.

In [ ]:
df = df.join(images)

df.head()

,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck,description,tags,header_image
app_id,,,,,,,,,,,,,,,
13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True,Enter the dark underworld of Prince of Persia ...,"[Action, Adventure, Parkour, Third Person, Gre...",https://cdn.akamai.steamstatic.com/steam/apps/...
113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True,Monaco: What's Yours Is Mine is a single playe...,"[Co-op, Stealth, Indie, Heist, Local Co-Op, St...",https://cdn.akamai.steamstatic.com/steam/apps/...
226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True,Escape Dead Island is a Survival-Mystery adven...,"[Zombies, Adventure, Survival, Action, Third P...",https://cdn.akamai.steamstatic.com/steam/apps/...
249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True,Dungeon of the Endless is a Rogue-Like Dungeon...,"[Roguelike, Strategy, Tower Defense, Pixel Gra...",https://cdn.akamai.steamstatic.com/steam/apps/...
250180,METAL SLUG 3,2015-09-14,True,False,False,Very Positive,90,5579,7.99,7.99,0.0,True,"“METAL SLUG 3”, the masterpiece in SNK’s emble...","[Arcade, Classic, Action, Co-op, Side Scroller...",https://cdn.akamai.steamstatic.com/steam/apps/...


Checking if there are any apps without image urls.

In [ ]:
df.header_image.isna().sum()

233

Filling in missing values.

In [ ]:
no_image_ids = df.loc[df.header_image.isna()].index

for id in no_image_ids:
    df.loc[id, "header_image"] = f"https://cdn.akamai.steamstatic.com/steam/apps/{id}/header.jpg"

### Saving Cleaned Datasets

Saving datasets for later use in applications.

In [ ]:
#developers column is added for future metadata based recommender
df = df.join(additions["developers"])

df = df[['title', 'date_release',
         'description', "tags", "developers",
         'rating', 'positive_ratio', 'user_reviews',
         'win', 'mac', 'linux', 'steam_deck',
         'price_final','header_image']].copy()

df.to_parquet("recommender_files/data/gameset.parquet")

## Building Knowledge-Based Recommender (Ranking by Genre)

In this first simple recommender the user will be asked to provide the genre. Recommender will filter the results for this genre and show top-25 best games based on review scores.

### Encoding Tags

In order to make this recommender work, `tags` column must be reshaped.

Each tag should be represented as a separate column with binary values, that show whether a game has this tag or not.

In [ ]:
df["tags"].head()

app_id
13500     [Action, Adventure, Parkour, Third Person, Gre...
113020    [Co-op, Stealth, Indie, Heist, Local Co-Op, St...
226560    [Zombies, Adventure, Survival, Action, Third P...
249050    [Roguelike, Strategy, Tower Defense, Pixel Gra...
250180    [Arcade, Classic, Action, Co-op, Side Scroller...
Name: tags, dtype: object

Now `tags` column contains lists with tags, so we need to transform this lists into separate columns and create one-hot encoded vector for each tag.

In [ ]:
tags = pd.get_dummies(df["tags"].explode()).groupby(level = 0).sum()

tags.head()

,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,3D Platformer,3D Vision,4 Player Local,4X,6DOF,8-bit Music,ATV,Abstract,Action,Action RPG,Action RTS,Action Roguelike,Action-Adventure,Addictive,Adventure,Agriculture,Aliens,Alternate History,Ambient,America,Animation & Modeling,Anime,Arcade,Archery,Arena Shooter,Artificial Intelligence,Assassin,Asymmetric VR,Asynchronous Multiplayer,Atmospheric,Audio Production,Auto Battler,Automation,Automobile Sim,BMX,Base Building,Baseball,Based On A Novel,Basketball,Battle Royale,Beat 'em up,Beautiful,Benchmark,Bikes,Blood,Board Game,Boss Rush,Bowling,Boxing,Building,Bullet Hell,Bullet Time,CRPG,Capitalism,Card Battler,Card Game,Cartoon,Cartoony,Casual,Cats,Character Action Game,Character Customization,Chess,Choices Matter,Choose Your Own Adventure,Cinematic,City Builder,Class-Based,Classic,Clicker,Co-op,Co-op Campaign,Coding,Cold War,Collectathon,Colony Sim,Colorful,Combat,Combat Racing,Comedy,Comic Book,Competitive,Conspiracy,Controller,Conversation,Cooking,Cozy,Crafting,Creature Collector,Cricket,Crime,Crowdfunded,Cult Classic,Cute,Cyberpunk,Cycling,Dark,Dark Comedy,Dark Fantasy,Dark Humor,Dating Sim,Deckbuilding,Demons,Design & Illustration,Destruction,Detective,Difficult,Dinosaurs,Diplomacy,Documentary,Dog,Dragons,Drama,Driving,Dungeon Crawler,Dungeons & Dragons,Dynamic Narration,Dystopian,Early Access,Economy,Education,Electronic,Electronic Music,Emotional,Epic,Episodic,Escape Room,Experience,Experimental,Exploration,FMV,FPS,Faith,Family Friendly,Fantasy,Farming,Farming Sim,Fast-Paced,Feature Film,Female Protagonist,Fighting,First-Person,Fishing,Flight,Football (American),Football (Soccer),Foreign,Fox,Free to Play,Funny,Futuristic,Gambling,Game Development,GameMaker,Games Workshop,Gaming,God Game,Golf,Gore,Gothic,Grand Strategy,Great Soundtrack,Grid-Based Movement,Gun Customization,Hack and Slash,Hacking,Hand-drawn,Hardware,Heist,Hentai,Hero Shooter,Hex Grid,Hidden Object,Historical,Hobby Sim,Hockey,Horror,Horses,Hunting,Idler,Illuminati,Immersive,Immersive Sim,Indie,Instrumental Music,Intentionally Awkward Controls,Interactive Fiction,Inventory Management,Investigation,Isometric,JRPG,Jet,Job Simulator,Jump Scare,Kickstarter,LEGO,LGBTQ+,Lemmings,Level Editor,Life Sim,Linear,Local Co-Op,Local Multiplayer,Logic,Loot,Looter Shooter,Lore-Rich,Lovecraftian,MMORPG,MOBA,Magic,Mahjong,Management,Mars,Martial Arts,Massively Multiplayer,Match 3,Mature,Mechs,Medical Sim,Medieval,Memes,Metroidvania,Military,Mini Golf,Minigames,Minimalist,Mining,Mod,Moddable,Modern,Motocross,Motorbike,Mouse only,Movie,Multiplayer,Multiple Endings,Music,Music-Based Procedural Generation,Musou,Mystery,Mystery Dungeon,Mythology,NSFW,Narration,Narrative,Nature,Naval,Naval Combat,Ninja,Noir,Nonlinear,Nostalgia,Nudity,Offroad,Old School,On-Rails Shooter,Online Co-Op,Open World,Open World Survival Craft,Otome,Outbreak Sim,Parkour,Parody,Party,Party Game,Party-Based RPG,Perma Death,Philosophical,Photo Editing,Physics,Pinball,Pirates,Pixel Graphics,Platformer,Point & Click,Political,Political Sim,Politics,Pool,Post-apocalyptic,Precision Platformer,Procedural Generation,Programming,Psychedelic,Psychological,Psychological Horror,Puzzle,Puzzle Platformer,PvE,PvP,Quick-Time Events,RPG,RPGMaker,RTS,Racing,Real Time Tactics,Real-Time,Real-Time with Pause,Realistic,Reboot,Relaxing,Remake,Replay Value,Resource Management,Retro,Rhythm,Robots,Rock Music,Roguelike,Roguelike Deckbuilder,Roguelite,Roguevania,Romance,Rome,Rugby,Runner,Sailing,Sandbox,Satire,Sci-fi,Science,Score Attack,Sequel,Sexual Content,Shoot 'Em Up,Shooter,Shop Keeper,Short,Side Scroller,Silent Protagonist,Simulation,Singleplayer,Skateboarding,Skating,Skiing,Sniper,Snooker,Snow,Snowboarding,Social Deduction,Software,Software Training,Sokoban,Solitaire,Souls-like,Soundtrack,Space,Space Sim,Spaceships,Spectacle fighter,Spelling,Split Screen,Sports,Stealth,Steam Machine,Steampunk,Story Rich,Strategy,Strategy RPG,Stylized,Submarine,Superhero,Supern

Adding new tags to the dataset and saving for further use.

In [ ]:
df = df.join(tags)

df.head()

,title,date_release,description,tags,developers,rating,positive_ratio,user_reviews,win,mac,linux,steam_deck,price_final,header_image,1980s,1990's,2.5D,2D,2D Fighter,2D Platformer,360 Video,3D,3D Fighter,3D Platformer,3D Vision,4 Player Local,4X,6DOF,8-bit Music,ATV,Abstract,Action,Action RPG,Action RTS,Action Roguelike,Action-Adventure,Addictive,Adventure,Agriculture,Aliens,Alternate History,Ambient,America,Animation & Modeling,Anime,Arcade,Archery,Arena Shooter,Artificial Intelligence,Assassin,Asymmetric VR,Asynchronous Multiplayer,Atmospheric,Audio Production,Auto Battler,Automation,Automobile Sim,BMX,Base Building,Baseball,Based On A Novel,Basketball,Battle Royale,Beat 'em up,Beautiful,Benchmark,Bikes,Blood,Board Game,Boss Rush,Bowling,Boxing,Building,Bullet Hell,Bullet Time,CRPG,Capitalism,Card Battler,Card Game,Cartoon,Cartoony,Casual,Cats,Character Action Game,Character Customization,Chess,Choices Matter,Choose Your Own Adventure,Cinematic,City Builder,Class-Based,Classic,Clicker,Co-op,Co-op Campaign,Coding,Cold War,Collectathon,Colony Sim,Colorful,Combat,Combat Racing,Comedy,Comic Book,Competitive,Conspiracy,Controller,Conversation,Cooking,Cozy,Crafting,Creature Collector,Cricket,Crime,Crowdfunded,Cult Classic,Cute,Cyberpunk,Cycling,Dark,Dark Comedy,Dark Fantasy,Dark Humor,Dating Sim,Deckbuilding,Demons,Design & Illustration,Destruction,Detective,Difficult,Dinosaurs,Diplomacy,Documentary,Dog,Dragons,Drama,Driving,Dungeon Crawler,Dungeons & Dragons,Dynamic Narration,Dystopian,Early Access,Economy,Education,Electronic,Electronic Music,Emotional,Epic,Episodic,Escape Room,Experience,Experimental,Exploration,FMV,FPS,Faith,Family Friendly,Fantasy,Farming,Farming Sim,Fast-Paced,Feature Film,Female Protagonist,Fighting,First-Person,Fishing,Flight,Football (American),Football (Soccer),Foreign,Fox,Free to Play,Funny,Futuristic,Gambling,Game Development,GameMaker,Games Workshop,Gaming,God Game,Golf,Gore,Gothic,Grand Strategy,Great Soundtrack,Grid-Based Movement,Gun Customization,Hack and Slash,Hacking,Hand-drawn,Hardware,Heist,Hentai,Hero Shooter,Hex Grid,Hidden Object,Historical,Hobby Sim,Hockey,Horror,Horses,Hunting,Idler,Illuminati,Immersive,Immersive Sim,Indie,Instrumental Music,Intentionally Awkward Controls,Interactive Fiction,Inventory Management,Investigation,Isometric,JRPG,Jet,Job Simulator,Jump Scare,Kickstarter,LEGO,LGBTQ+,Lemmings,Level Editor,Life Sim,Linear,Local Co-Op,Local Multiplayer,Logic,Loot,Looter Shooter,Lore-Rich,Lovecraftian,MMORPG,MOBA,Magic,Mahjong,Management,Mars,Martial Arts,Massively Multiplayer,Match 3,Mature,Mechs,Medical Sim,Medieval,Memes,Metroidvania,Military,Mini Golf,Minigames,Minimalist,Mining,Mod,Moddable,Modern,Motocross,Motorbike,Mouse only,Movie,Multiplayer,Multiple Endings,Music,Music-Based Procedural Generation,Musou,Mystery,Mystery Dungeon,Mythology,NSFW,Narration,Narrative,Nature,Naval,Naval Combat,Ninja,Noir,Nonlinear,Nostalgia,Nudity,Offroad,Old School,On-Rails Shooter,Online Co-Op,Open World,Open World Survival Craft,Otome,Outbreak Sim,Parkour,Parody,Party,Party Game,Party-Based RPG,Perma Death,Philosophical,Photo Editing,Physics,Pinball,Pirates,Pixel Graphics,Platformer,Point & Click,Political,Political Sim,Politics,Pool,Post-apocalyptic,Precision Platformer,Procedural Generation,Programming,Psychedelic,Psychological,Psychological Horror,Puzzle,Puzzle Platformer,PvE,PvP,Quick-Time Events,RPG,RPGMaker,RTS,Racing,Real Time Tactics,Real-Time,Real-Time with Pause,Realistic,Reboot,Relaxing,Remake,Replay Value,Resource Management,Retro,Rhythm,Robots,Rock Music,Roguelike,Roguelike Deckbuilder,Roguelite,Roguevania,Romance,Rome,Rugby,Runner,Sailing,Sandbox,Satire,Sci-fi,Science,Score Attack,Sequel,Sexual Content,Shoot 'Em Up,Shooter,Shop Keeper,Short,Side Scroller,Silent Protagonist,Simulation,Singleplayer,Skateboarding,Skating,Skiing,Sniper,Snooker,Snow,Snowboarding,Social Deduction,Software,Software Training,Sokoban,Solitaire,Souls-like,Soundtrack,Space,Space Sim,Spaceships,Spectacle fight

In [ ]:
df.to_parquet("recommender_files/data/gameset_ohe.parquet")

### Checking results

We can build our recommender in form of the table and check it's work.

Some of supported genres are:

`Action, Arcade, Rhythm, Fighting, First-Person, Third-Person, Hack and Slash, Platformer, Runner, Shooter, Bullet Hell, Adventure, Casual, Hidden Object, Metroidvania, Puzzle, Story Rich, Visual Novel, RPG, Adventure, Roguelike, Strategy, Simulation, Dating Sim, Immersive, Strategy, Card Game, Board Game, City Builder, Military, Tower Defense, Sports, Racing`

*In case you want to try yourself.*

In [ ]:
# enter any genre here
genre = "Action"

import pandas as pd
df = pd.read_parquet("recommender_files/data/gameset_ohe.parquet")

table = df.loc[(df[genre] == 1) & (df["rating"] == "Overwhelmingly Positive") |
       (df["rating"] == "Very Positive") |
       (df["rating"] == "Positive")].sort_values(by =
                                                ["user_reviews",
                                                 "positive_ratio"],
                                                 ascending = False)[:25]

table.head(25)

,title,date_release,description,tags,developers,rating,positive_ratio,user_reviews,win,mac,...,Well-Written,Werewolves,Western,Wholesome,Word Game,World War I,World War II,Wrestling,Zombies,eSports
app_id,,,,,,,,,,,,,,,,,,,,,
304930,Unturned,2017-07-07,You're a survivor in the zombie infested ruins...,"[Free to Play, Survival, Zombies, Multiplayer,...",[Smartly Dressed Games],Very Positive,91,515016,True,True,...,0,0,0,0,0,0,0,0,1,0
1281930,tModLoader,2020-05-16,"Dig, Fight, and Build your way through the wor...","[Adventure, Action, Free to Play, Indie, RPG, ...",None,Overwhelmingly Positive,97,188684,True,True,...,0,0,0,0,0,0,0,0,0,0
319630,Life is Strange - Episode 1,2015-01-29,Episode 1 now FREE! Life is Strange is an awar...,"[Story Rich, Choices Matter, Great Soundtrack,...","[DONTNOD Entertainment, Feral Interactive (Mac...",Overwhelmingly Positive,96,160660,True,True,...,0,0,0,0,0,0,0,0,0,0
700330,SCP: Secret Laboratory,2017-12-29,Deep within the SCP Foundation during a contai...,"[Free to Play, Horror, Multiplayer, First-Pers...",[Northwood Studios],Very Positive,91,154538,True,False,...,0,0,0,0,0,0,0,0,0,0
1782210,Crab Game,2021-10-29,Crab Game is a First-Person Multiplayer game w...,"[Psychological Horror, Multiplayer, Free to Pl...",[Dani],Very Positive,92,144524,True,True,...,0,0,0,0,0,0,0,0,0,0
1625450,Muck,2021-06-05,Muck is a survival-roguelike. Collect resource...,"[Survival, Crafting, Multiplayer, Building, Ro...",[Dani],Very Positive,94,139841,True,True,...,0,0,0,0,0,0,0,0,0,0
10,Counter-Strike,2000-11-01,Play the world's number 1 online action game. ...,"[Action, FPS, Multiplayer, Shooter, Classic, T...",[Valve],Overwhelmingly Positive,97,136118,True,True,...,0,0,0,0,0,0,0,0,0,1
203160,Tomb Raider,2013-03-04,Tomb Raider explores the intense origin story ...,"[Adventure, Action, Female Protagonist, Third ...","[Crystal Dynamics, Eidos-Montréal, Feral Inter...",Overwhelmingly Positive,96,136055,True,True,...,0,0,0,0,0,0,0,0,0,0
220,Half-Life 2,2007-10-10,1998. HALF-LIFE sends a shock through the game...,"[FPS, Action, Sci-fi, Classic, Singleplayer, S...",[Valve],Overwhelmingly Positive,97,122230,True,True,...,0,0,0,0,0,0,0,0,1,0


### Running the app

Creating code for the application.

In short, this are the steps performed in the code cell:
1. We create a form where the user can type their preferred genre, or select it from (very long) list of possible options and than request the recommendations.
    - If user did not chose the genre and requested recommendations we'll kindly ask them again to choose one.
2. Than we filter the dataset so it contains only apps with "Positive" and higher ratings, sort values by *number of user reviews* and *positive reviews ratio* in descending order and crop this sorted dataset to top 25 data entries.
    - It is done so because it's important, whether an app with 1000 reviews has a big positive reviews ratio or an app with 10 reviews has a big positive reviews ratio.
3. Finally, we show selected apps to the user, along with their rating, image, release date and supported platforms.

In [ ]:
%%writefile recommender_files/kb_recommender_by_genre.py

def top_25_games():
    import streamlit as st
    import pandas as pd

    df = pd.read_parquet("recommender_files/data/gameset_ohe.parquet")

    with st.form("my_form"):

        st.title("Top-25 games by Genre")

        # creating a form to select a genre
        genre = st.selectbox("Genre:",
                        options = df.columns[14:],
                        index = None,
                        placeholder = "Start typing to search among options")
        st.write('You selected:', genre)


        submitted = st.form_submit_button("Recommend")

        # kindly ask for genre if none was provided
        if submitted:
            if genre == None:
                st.write("Please, consider choosing a genre")
            # proceed if genre was provided
            else:
                # filtering the dataset and sorting values in descending order
                table = df.loc[(df[genre] == 1) &
                               (df["rating"] == "Overwhelmingly Positive") |
                                (df["rating"] == "Very Positive") |
                                (df["rating"] == "Positive")].sort_values(by =
                                                               ["user_reviews",
                                                                "positive_ratio"],
                                                                ascending = False)[:25]

                platforms = ["Windows", "MacOS", "Linux", "SteamDeck"]

                # showing top 25 games from sorted dataset
                for index in table.index:

                    supported_platforms = list(df.loc[index, ["win", "mac", "linux", "steam_deck"]])
                    supported_platforms = list(zip(platforms, supported_platforms))
                    supported_platforms = [i[0] for i in supported_platforms if i[1] == True]


                    # creating an information card for each of the games
                    with st.container():

                        st.header(table.loc[index, "title"], divider = "rainbow")

                        col1, col2 = st.columns([1, 2])

                        with col1:
                            st.image(table.loc[index, "header_image"])

                        with col2:
                            st.subheader(f"_Rating_: :green[ {table.loc[index, 'rating']} ]")
                            st.text(f"Reliase date: {table.loc[index, 'date_release']}")
                            st.text(("Supported plarforms: " + ", ".join(supported_platforms)))

top_25_games()

Overwriting recommender_files/kb_recommender_by_genre.py


You can run the app by running the code in the next cell.

To enter the server with the app, please:
1. Copy the numbers in the first line of the output.
    - ![1.jpeg](https://i.ibb.co/t4BHH52/1.jpg")
2. Click on the generated link in the very bottom of the output.
    - ![2.jpeg](https://i.ibb.co/528T9mv/2.jpg)
3. Enter copied numbers in the text field and click submit.
    - ![3.jpeg](https://i.ibb.co/d6Z3fs8/3.jpg)
4. After closing the application tab, don't forget to stop the cell execution.
    - ![4.jpeg](https://i.ibb.co/2FJ7L62/4.jpg)

In [ ]:
!pip install streamlit &> /dev/null
!npm install localtunnel &> /dev/null

!streamlit run recommender_files/kb_recommender_by_genre.py & >/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

## Content-Based Recommender Using Title Descriptions

In [ ]:
# restarting the Colab kernel to free all RAM
import os
os._exit(00)

In [ ]:
import numpy as np
import pandas as pd

### Loading Saved Dataset


Loading the dataset without one-hot encoded tags.

In [ ]:
df = pd.read_parquet("recommender_files/data/gameset.parquet")

For this recommender we'll pick apps with ratings from "Mixed" to "Overwhelmingly Positive" and sort the data in descending order again, because we'll need to crop it later.

In [ ]:
# ts stands for truncated set

ts = df.loc[(df["rating"] == "Overwhelmingly Positive") |
            (df["rating"] == "Very Positive") |
            (df["rating"] == "Positive") |
            (df["rating"] == "Mostly Positive") |
            (df["rating"] == "Mixed")].sort_values(by =
                                                  ["user_reviews",
                                                   "positive_ratio"],
                                                   ascending = False)

ts = ts["description"].copy()

ts.head()

app_id
304930     You're a survivor in the zombie infested ruins...
433850     Z1 Battle Royale is a Free to Play, fast-paced...
1281930    Dig, Fight, and Build your way through the wor...
1468810    Tale of Immortal is an open-world sandbox base...
698780     The Literature Club is full of cute girls! Wil...
Name: description, dtype: object

### Creating Document Vectors

For this recommender we'll need to measure how similar are descriptions of different games to one another.
Cosine similarity can help us to find that measure, but before computing it we need to convert text of the descriptions into vectors of numbers.

TF-IDF Vectorizer is suited perfectly for this case because we don't want to give a lot of weight to the words that appear very often - we would rather assign more value to the words that are specific to the description of each particular game.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents = "unicode", stop_words = "english")

Converting descriptions into vectors.

***! Note on cropping***

Computing cosine similarity is computationally quite expensive - most importantly, it takes a lot of RAM.

The dataset is cropped to top 31500 data entries so it could fit in 12.7 GB of (free) Colab RAM.

(*If you have more RAM feel free to remove this and any further slicing and use the whole dataset*).

In [ ]:
tfidf_matrix = tfidf.fit_transform(ts[:31500])

### Computing Cosine Similarities.

Since previously we used TF-IDF Vectorizer on descriptions, computing simpler and less expensive linear kernel will suffice.

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

cosine_similarity = linear_kernel(tfidf_matrix)

Converting cosine similarity matrix into a dataset along with app ids, for easier reference.

In [ ]:
# don't forget to remove slicing, in case of computing cosine similarity for the whole dataset
csf = pd.DataFrame(list(zip(ts[:31500].index, cosine_similarity)))
csf.columns = ["app_id", "cosine_similarity"]
csf = csf.set_index("app_id")

Saving dataset for later use in the application.

(*In case you have computed cosine similarity for the whole dataset, you can uncomment and run the code in the code cell bellow to save your dataset. You can also save and then load it as one file, but it will slow down execution of the app.*)

In [ ]:
# csf[:7875].to_parquet("recommender_files/cosine similarities/descriptions/cosine_similarity.parquet")
# csf[7875:15750].to_parquet("recommender_files/cosine similarities/descriptions/cosine_similarity2.parquet")
# csf[15750:23625].to_parquet("recommender_files/cosine similarities/descriptions/cosine_similarity3.parquet")
# csf[23625:].to_parquet("recommender_files/cosine similarities/descriptions/cosine_similarity4.parquet")

### Checking results

We can look at the results in form of the table.

*Change the value of the `game_title` variable, in case you want to try yourself, but be careful - selected title might not be in the dataset, since there are less data entries now.*

(*If kernel shuts down just run the cell one more time.*)

In [ ]:
# enter game title here
game_title = "Prince of Persia: Warrior Within™"

import pandas as pd
df = pd.read_parquet("recommender_files/data/gameset.parquet")

csf = pd.read_parquet("recommender_files/cosine similarities/descriptions/cosine_similarity.parquet")
for i in range(2, 5):
    csf = pd.concat([csf, pd.read_parquet(f"recommender_files/cosine similarities/descriptions/cosine_similarity{i}.parquet")])


ts = df.loc[(df["rating"] == "Overwhelmingly Positive") |
            (df["rating"] == "Very Positive") |
            (df["rating"] == "Positive") |
            (df["rating"] == "Mostly Positive") |
            (df["rating"] == "Mixed")].sort_values(by =
                                                  ["user_reviews",
                                                   "positive_ratio"],
                                                   ascending = False)

ts = ts[["title", "date_release", "rating", "win", "mac",
         "linux", "steam_deck", "price_final", "header_image"]][:31500]

game_id = ts.loc[ts["title"] == game_title].index

cosine_similarity = csf.loc[game_id, "cosine_similarity"].values[0]
cosine_similarity = list(zip(cosine_similarity, csf.index))
cosine_similarity = sorted(cosine_similarity, reverse = True)

ids = [i[1] for i in cosine_similarity[1:26]]

table = ts.loc[ids]

display(table.head(25))

del([df, csf, ts, game_id, cosine_similarity, ids, table])

,title,date_release,rating,win,mac,linux,steam_deck,price_final,header_image
app_id,,,,,,,,,
19980,Prince of Persia®,2008-12-10,Mostly Positive,True,False,False,True,9.99,https://cdn.akamai.steamstatic.com/steam/apps/...
33320,Prince of Persia: The Forgotten Sands™,2010-06-08,Mixed,True,False,False,True,9.99,https://cdn.akamai.steamstatic.com/steam/apps/...
621080,Persian Nights: Sands of Wonders,2017-06-29,Very Positive,True,True,True,True,9.99,https://cdn.akamai.steamstatic.com/steam/apps/...
13530,Prince of Persia: The Two Thrones™,2008-11-21,Mostly Positive,True,False,False,True,9.99,https://cdn.akamai.steamstatic.com/steam/apps/...
1550840,RATUZ,2022-04-22,Very Positive,True,False,False,True,5.99,https://cdn.akamai.steamstatic.com/steam/apps/...
499100,Dark Parables: The Exiled Prince Collector's E...,2016-08-09,Very Positive,True,False,False,True,9.99,https://cdn.akamai.steamstatic.com/steam/apps/...
13600,Prince of Persia®: The Sands of Time,2008-11-21,Very Positive,True,False,False,True,9.99,https://cdn.akamai.steamstatic.com/steam/apps/...
545830,Princess of Tavern Collector's Edition,2017-10-23,Mixed,True,True,False,True,9.99,https://cdn.akamai.steamstatic.com/steam/apps/...
1846940,There Won't be Light,2022-09-25,Mixed,True,False,False,True,1.99,https://cdn.akamai.steamstatic.com/steam/apps/...


### Running the app

Creating code for the application.

Steps performed in the code cell:
1. We load the dataset and crop it to top 31500 values, so it matches the size of cosine similarity matrix.
2. We create a form where the user can select desired game title and request the recommendations.
    - If user did not chose the game title and requested recommendations we'll kindly ask them again to choose it.
3. Than we load the dataset with cosine similarities.
4. We obtain an id (index) of selected game title and by this id we locate a similarity vector for that game in cosine similarities dataset.
5. We combine similarity vector with game title ids and sort this list in descending order.
6. Than we pick top 25 ids from this list, excluding the first element, because it is the chosen game title itself (with similarity measure of 1).
7. We locate the titles with the highest similarity scores by their ids in our dataset with games.
8. Finally, we show selected games to the user, along with their rating, image, release date and supported platforms.

In [ ]:
%%writefile recommender_files/cb_recommender_by_description.py

def top_25_similar_games():
    import streamlit as st
    import pandas as pd

    # in this app we'll use dataset without encoded tags
    df = pd.read_parquet("recommender_files/data/gameset.parquet")

    # cropping the dataset
    ts = df.loc[(df["rating"] == "Overwhelmingly Positive") |
                (df["rating"] == "Very Positive") |
                (df["rating"] == "Positive") |
                (df["rating"] == "Mostly Positive") |
                (df["rating"] == "Mixed")].sort_values(by =
                                                      ["user_reviews",
                                                       "positive_ratio"],
                                                       ascending = False)

    ts = ts[["title", "date_release", "rating", "win", "mac",
             "linux", "steam_deck", "price_final", "header_image"]][:31500]

    # creating a form to select an game title
    with st.form("my_form"):

        st.title("Top-25 similar games")

        game_title = st.selectbox("Game Title:",
                        options = ts["title"].values,
                        index = None,
                        placeholder = "Start typing to search among options")
        st.write('You selected:', game_title)


        submitted = st.form_submit_button("Recommend")

        if submitted:
            # kindly ask for game title if none was provided
            if game_title == None:
                st.write("Please, consider choosing a game title")
            # proceed if game title was provided
            else:
                # loading dataset with cosine similarities
                csf = pd.read_parquet("recommender_files/cosine similarities/descriptions/cosine_similarity.parquet")
                for i in range(2,5):
                    csf = pd.concat([csf, pd.read_parquet(f"recommender_files/cosine similarities/descriptions/cosine_similarity{i}.parquet")])

                # obtaining app id of chosen game title
                game_id = ts.loc[ts["title"] == game_title].index

                # locating a similarity vector by app id
                cosine_similarity = csf.loc[game_id, "cosine_similarity"].values[0]
                # combining similarity vector with app ids and sorting the list
                cosine_similarity = list(zip(cosine_similarity, csf.index))
                cosine_similarity = sorted(cosine_similarity, reverse = True)

                # choosing app ids of top 25 games
                ids = [i[1] for i in cosine_similarity[1:26]]

                # locating games by app ids in the dataset
                table = ts.loc[ids]

                platforms = ["Windows", "MacOS", "Linux", "SteamDeck"]

                # showing top 25 games from sorted dataset
                for index in table.index:

                    supported_platforms = list(df.loc[index, ["win", "mac", "linux", "steam_deck"]])
                    supported_platforms = list(zip(platforms, supported_platforms))
                    supported_platforms = [i[0] for i in supported_platforms if i[1] == True]

                    # creating an information card for each of the games
                    with st.container():

                        st.header(table.loc[index, "title"], divider = "rainbow")

                        col1, col2 = st.columns([1, 2])

                        with col1:
                            st.image(table.loc[index, "header_image"])

                        with col2:
                            if table.loc[index, 'rating'] == "Mixed":
                                st.subheader(f"_Rating_: :orange[{table.loc[index, 'rating']}]")
                            else:
                                st.subheader(f"_Rating_: :green[{table.loc[index, 'rating']}]")
                            st.text(f"Reliase date: {table.loc[index, 'date_release']}")
                            st.text(("Supported plarforms: " + ", ".join(supported_platforms)))
                            if table.loc[index, 'price_final'] == 0.0:
                                st.text(f"Price: Free")
                            else:
                                st.text(f"Price: {table.loc[index, 'price_final']}$")

top_25_similar_games()

Overwriting recommender_files/cb_recommender_by_description.py


You can run the app by running the code in the next two cells. *(Please, be patient and wait while the first cell will finish it's execution.)*

***! This application will take ~1 minute to load the results.***

In [ ]:
# restarting the Colab kernel to free all RAM
import os
os._exit(00)

In [ ]:
!pip install streamlit &> /dev/null
!npm install localtunnel &> /dev/null

!streamlit run recommender_files/cb_recommender_by_description.py & >/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

## Content-Based Recommender Using Title Metadata

In [ ]:
# restarting the Colab kernel to free all RAM
import os
os._exit(00)

In [ ]:
import numpy as np
import pandas as pd

### Loading Saved Dataset

In [ ]:
mds = pd.read_parquet("recommender_files/data/gameset.parquet")

In this case we are going to recommend games only with positive reviews, because cosine similarity is even more computationally expensive than linear kernel, used before.

In [ ]:
# mdts stands for metadata truncated set
mdts = mds.loc[(mds["rating"] == "Overwhelmingly Positive") |
               (mds["rating"] == "Very Positive") |
               (mds["rating"] == "Positive") |
               (mds["rating"] == "Mostly Positive")].sort_values(by =
                                                                ["user_reviews",
                                                                "positive_ratio"],
                                                                ascending = False)

mdts.head()

,title,date_release,description,tags,developers,rating,positive_ratio,user_reviews,win,mac,linux,steam_deck,price_final,header_image
app_id,,,,,,,,,,,,,,
304930,Unturned,2017-07-07,You're a survivor in the zombie infested ruins...,"[Free to Play, Survival, Zombies, Multiplayer,...",[Smartly Dressed Games],Very Positive,91,515016,True,True,True,True,0.00,https://cdn.akamai.steamstatic.com/steam/apps/...
1281930,tModLoader,2020-05-16,"Dig, Fight, and Build your way through the wor...","[Adventure, Action, Free to Play, Indie, RPG, ...",None,Overwhelmingly Positive,97,188684,True,True,True,True,0.00,https://cdn.akamai.steamstatic.com/steam/apps/...
698780,Doki Doki Literature Club!,2017-10-06,The Literature Club is full of cute girls! Wil...,"[Psychological Horror, Visual Novel, Anime, Fr...",[Team Salvato],Overwhelmingly Positive,96,184949,True,True,False,True,0.00,https://cdn.akamai.steamstatic.com/steam/apps/...
319630,Life is Strange - Episode 1,2015-01-29,Episode 1 now FREE! Life is Strange is an awar...,"[Story Rich, Choices Matter, Great Soundtrack,...","[DONTNOD Entertainment, Feral Interactive (Mac...",Overwhelmingly Positive,96,160660,True,True,True,True,19.99,https://cdn.akamai.steamstatic.com/steam/apps/...
700330,SCP: Secret Laboratory,2017-12-29,Deep within the SCP Foundation during a contai...,"[Free to Play, Horror, Multiplayer, First-Pers...",[Northwood Studios],Very Positive,91,154538,True,False,False,True,0.00,https://cdn.akamai.steamstatic.com/steam/apps/...


Checking if any games have missing values in "developers" column.

In [ ]:
mdts["developers"].isna().sum()

185

Looks like, there are 185 missing values.

Living 185 potentially good games behind is not very nice, so lets find them.

After two hours of laborious work all missing developers were found.

*party*

In [ ]:
missing_developers = [
["TML Team"],
["(Hong Kong) GKD"],
["Prism Game Studios Ltd."],
["Gearbox Software", "Blind Squirrel Games"],
["Laser Unicorns"],
["PORTANIS"],
["Breadmen"],
["The Chronicles Of Myrtana Team"],
["UL Solutions"],
["SureAI"],
["Valve"],
["CAPCOM Co., Ltd."],
["KINODOM PRODUCTIONS"],
["empty"],
["Breadman"],
["Alan Edwardes"],
["Filip Victor", "Valve"],
["Vortal Storm"],
["Odd Time Studios", "Aperture Tag Team", "Rockhopper Studios"],
["empty"],
["SureAI"],
["Oats Studios"],
["Stridemann"],
["Source VR Mod Team"],
["FireFly Studios"],
["Croteam"],
["One More Level", "3D Realms", "Slipgate Ironworks™", "All in! Games"],
["Future Games Select, Cayle George"],
["marnamai"],
["STUDIO RADI-8"],
["Table 9 Studio"],
["empty"],
["TEAM JBMOD"],
["Caustic Creative"],
["Tripwire Interactive, Type AB"],
["Blitzkrieg Mod Team"],
["empty"],
["Galactic Cafe"],
["Bohemia Interactive"],
["SnowDropEscape development team"],
["RTK Entertainment"],
["Chuck Wilson"],
["empty"],
["Spike Chunsoft Co., Ltd."],
["Masangsoft"],
["Frere d'Arme"],
["Prism Game Studios Ltd."],
["Team ModCom"],
["Fortress Forever Development Team", "Trepid Studios"],
["Archaic Entertainment"],
["MMod Team"],
["OVERKILL - a Starbreeze Studio.", "Payday Productions"],
["Darklight Games"],
["Team Jabroni"],
["The SKSE Team"],
["Relic Entertainment", "Europe at War Development Team"],
["Troiz"],
["Wolfmap", "RTCWCoop Team", "Dark Matter Productions"],
["empty"],
["Benjamin Bryant"],
["Vect0R"],
["SureAI"],
["ichbinhamma"],
["Elseware Experience"],
["Evac Dev. Team"],
["Relic Entertainment", "The Great War 1918 Development Team"],
["KONAMI"],
["Bitl Development Studio"],
["Wevr, Inc."],
["Tripwire Interactive", "Kernow Interactive"],
["Corridor Digital", "Ubisoft Entertainment"],
["empty"],
["Rusty Lake", "The LAB"],
["Bohemia Interactive"],
["NetherRealm Studios", "Warner Bros. Interactive Entertainment"],
["EXOR Studios"],
["empty"],
["HLA team"],
["TechSmith Corporation"],
["yeo"],
["Philipp Lehner"],
["Crystice Softworks"],
["Kirill Bulavintsev"],
["Blendo Games"],
["iDragons creative studio"],
["ONEONE1"],
["Eric Schafenberg"],
["PLut"],
["Roroco Studio"],
["Telltale Games"],
["Relic Entertainment", "Modern Combat Development Team", "Black Sand Studios"],
["Reaktor"],
["Source VR Mod Team"],
["Black Chicken Studios, inc."],
["From Earth Team"],
["KINODOM PRODUCTIONS"],
["NetherRealm Studios", "Warner Bros. Interactive Entertainment"],
["Astrabit Studios"],
["Infogrames"],
["VisualArts/Key"],
["WIZNWAR", "FLARB LLC"],
["Running With Scissors"],
["YellowAfterlife"],
["JaviteSoft"],
["Edmund McMillen and Florian Himsl"],
["Volition"],
["empty"],
["Tim Bergholz - Chamfer Zone"],
["Source VR Mod Team"],
["Ubisoft Montreal"],
["Sprouting Potato"],
["KOEI TECMO GAMES CO., LTD."],
["MadLife Divertissement"],
["Telltale Games"],
["Sandstorm Productions"],
["Heath Games"],
["Warhorse Studios"],
["TashiKani"],
["Digital Paintball"],
["KLRU-TV", "Austin PBS"],
["Bethesda Game Studios"],
["Battle of Crete Development Team"],
["Infogrames"],
["Leikir Studio"],
["Microsoft Studios"],
["Fuzzy Ghost"],
["Valve"],
["Warner Bros."],
["Ignishot"],
["IDEA FACTORY", "Frontier Works"],
["Lunastellia", "Matux"],
["Nitreal", "GameHouse"],
["Gentlymad Studios"],
["Infogrames"],
["oneoreight", "Team EARTHWARS"],
["Schmidt Workshops"],
["CREATIVE ASSEMBLY"],
["Team Reliquary ltd."],
["橘喵喵"],
["Crank Goblin"],
["Lucas Pope and Keiko Ishizaka"],
["Warhorse Studios"],
["Noclip Video"],
["Valve"],
["Grab Games"],
["Subverse Productions"],
["Bohemia Interactive"],
["Matthew Bradshaw"],
["Insurgency Development Team"],
["Team Chivalry"],
["Stefan Kraus", "Timo Armbruster"],
["Valve"],
["Supergiant Games"],
["SMG Studio"],
["RO2/RS Community", "Tripwire Interactive"],
["The Game Bakers"],
["CAPCOM CO., LTD."],
["Devolver Digital"],
["Raw Fury"],
["Endless Loop Studios", "Code Monkey"],
["Cold Beam Games"],
["RO2/RS Community", "Tripwire Interactive"],
["Wargaming Group Limited"],
["Geeta Games"],
["Dr BLOC"],
["Bohemia Interactive"],
["MicroProse Software, Inc"],
["谭浩翔"],
["Double Fine Productions"],
["Atomic Wolf", "L.INC"],
["Daedalic Entertainment"],
["These Are The Good Games"],
["ChamferZone"],
["Wargaming Group Limited"],
["Adam Foster"],
["Mighty Rocket Studio"],
["The Blank Canvas", "Lobo"],
["ChamferZone"],
["empty"],
["Mixamo"],
["Yanka Prod"],
["Clouds and Blue"],
["One Hamsa"],
["NEKO WORKs"],
["Gone Coyote"]
]

In [ ]:
missing_developers_ids = mdts.loc[mdts["developers"].isna()].index

Adding missing developers to the dataset.

In [ ]:
for entry in zip(missing_developers_ids, missing_developers):
    mdts.at[entry[0], "developers"] = entry[1]

Dropping the columns with empty entries.

In [ ]:
empty_developers_ids = list()
for index, value in zip(mdts["developers"].index, mdts["developers"]):
    if len(value) == 0:
        empty_developers_ids.append(index)

In [ ]:
mdts = mdts.drop(index = empty_developers_ids)

### Creating Metadata Column

Removing whitespaces in developer's names and in tags, so they could be recognized as one-word entries.

In [ ]:
def preprocess_text(x):
    return [str.lower(i.replace(" ", "")) for i in x]

In [ ]:
for feature in ["developers", "tags"]:
    mdts[feature] = mdts[feature].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

Joining values from `developers` and `tags` columns into new `metadata` column.

In [ ]:
mdts["metadata"] = mdts.apply(lambda x: " ".join(x["developers"]) +
                                        " " +
                                        " ".join(x["tags"]), axis = 1)

In [ ]:
mdts["metadata"][:1].values

array(['smartlydressedgames freetoplay survival zombies multiplayer openworld co-op crafting sandbox adventure shooter first-person post-apocalyptic lootershooter action fps singleplayer massivelymultiplayer atmospheric indie casual'],
      dtype=object)

Saving dataset for further use in the application.

In [ ]:
mdts.to_parquet("recommender_files/data/gameset_with_metadata.parquet")

### Creating Document Vectors

For this case we are going to use Count Vectorizer, because, unlike previously, we don't want to put more emphasis on rare cases (like developer, or very specific genre).

***! Note on cropping***

The dataset is cropped to top 25500 data entries so it could fit in 12.7 GB of (free) Colab RAM.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cvec = CountVectorizer(strip_accents = "unicode", stop_words = "english")

mdts = mdts["metadata"].copy()
count_matrix = cvec.fit_transform(mdts[:25500])

Computing cosine similarity matrix, storing it into the dataset and saving it for later use in the application.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim = cosine_similarity(count_matrix)

In [ ]:
# stands for meta cosine similarity
mcs = pd.DataFrame(list(zip(mdts[:25500].index, cos_sim)))
mcs.columns = ["app_id", "cosine_similarity"]
mcs = mcs.set_index("app_id")

In [ ]:
# mcs[:6375].to_parquet("recommender_files/cosine similarities/meta/cos_sim.parquet")
# mcs[6375:12750].to_parquet("recommender_files/cosine similarities/meta/cos_sim2.parquet")
# mcs[12750:19125].to_parquet("recommender_files/cosine similarities/meta/cos_sim3.parquet")
# mcs[19125:].to_parquet("recommender_files/cosine similarities/meta/cos_sim4.parquet")

### Checking results

(*If kernel shuts down just run the cell one more time.*)

In [ ]:
# enter app title here (in case you want to try yourself)
game_title = "Prince of Persia: Warrior Within™"

import pandas as pd
mds = pd.read_parquet("recommender_files/data/gameset.parquet")

mcs = pd.read_parquet("recommender_files/cosine similarities/meta/cos_sim.parquet")
for i in range(2,5):
    mcs = pd.concat([mcs, pd.read_parquet(f"recommender_files/cosine similarities/meta/cos_sim{i}.parquet")])


mdts = mds.loc[(mds["rating"] == "Overwhelmingly Positive") |
               (mds["rating"] == "Very Positive") |
               (mds["rating"] == "Positive") |
               (mds["rating"] == "Mostly Positive")].sort_values(by =
                                                                ["user_reviews",
                                                                 "positive_ratio"],
                                                                 ascending = False)

mdts = mdts[["title", "date_release", "rating", "win", "mac",
             "linux", "steam_deck", "price_final", "header_image"]][:25500]


game_id = mdts.loc[mdts["title"] == game_title].index

cosine_similarity = mcs.loc[game_id, "cosine_similarity"].values[0]
cosine_similarity = list(zip(cosine_similarity, mcs.index))
cosine_similarity = sorted(cosine_similarity, reverse = True)

ids = [i[1] for i in cosine_similarity[1:26]]

table = mdts.loc[ids]

display(table.head(25))

del([mds, mcs, mdts, game_id, cosine_similarity, ids, table])

,title,date_release,rating,win,mac,linux,steam_deck,price_final,header_image
app_id,,,,,,,,,
13600,Prince of Persia®: The Sands of Time,2008-11-21,Very Positive,True,False,False,True,9.99,https://cdn.akamai.steamstatic.com/steam/apps/...
13530,Prince of Persia: The Two Thrones™,2008-11-21,Mostly Positive,True,False,False,True,9.99,https://cdn.akamai.steamstatic.com/steam/apps/...
19980,Prince of Persia®,2008-12-10,Mostly Positive,True,False,False,True,9.99,https://cdn.akamai.steamstatic.com/steam/apps/...
224920,Legacy of Kain: Soul Reaver,1999-09-08,Mostly Positive,True,False,False,True,0.00,https://cdn.akamai.steamstatic.com/steam/apps/...
388410,Darksiders II Deathinitive Edition,2015-11-05,Mostly Positive,True,False,False,True,29.99,https://cdn.akamai.steamstatic.com/steam/apps/...
606280,Darksiders III,2018-11-27,Mostly Positive,True,False,False,True,39.99,https://cdn.akamai.steamstatic.com/steam/apps/...
224300,Legacy of Kain: Defiance,2012-12-06,Very Positive,True,False,False,True,6.99,https://cdn.akamai.steamstatic.com/steam/apps/...
7000,Tomb Raider: Legend,2007-03-29,Very Positive,True,False,False,True,6.99,https://cdn.akamai.steamstatic.com/steam/apps/...
245280,ENSLAVED™: Odyssey to the West™ Premium Edition,2013-10-24,Very Positive,True,False,False,True,19.99,https://cdn.akamai.steamstatic.com/steam/apps/...


### Running the app

This application works by the same scenario as previous one.

It differs from previous only in size ot the dataset, absence of reviews with "Mixed" rating and different values in similarity vectors.

In [ ]:
%%writefile recommender_files/cb_recommender_by_metadata.py

def top_25_similar_games_by_meta():
    import streamlit as st
    import pandas as pd

    # in this app we'll use pre-saved dataset with metadata
    mds = pd.read_parquet("recommender_files/data/gameset_with_metadata.parquet")

    # cropping the dataset
    mdts = mds.loc[(mds["rating"] == "Overwhelmingly Positive") |
                   (mds["rating"] == "Very Positive") |
                   (mds["rating"] == "Positive") |
                   (mds["rating"] == "Mostly Positive")].sort_values(by =
                                                                    ["user_reviews",
                                                                     "positive_ratio"],
                                                                     ascending = False)

    mdts = mdts[["title", "date_release", "rating", "win", "mac",
             "linux", "steam_deck", "price_final", "header_image"]][:25500]

    # creating a form to select an game title
    with st.form("my_form"):

        st.title("Top-25 similar games (by metadata)")

        game_title = st.selectbox("Game Title:",
                        options = mdts["title"].values,
                        index = None,
                        placeholder = "Start typing to search among options")
        st.write('You selected:', game_title)


        submitted = st.form_submit_button("Recommend")

        if submitted:
            # kindly ask for game title if none was provided
            if game_title == None:
                st.write("Please, consider choosing a game title")
            # proceed if game title was provided
            else:
                # loading dataset with cosine similarities
                mcs = pd.read_parquet("recommender_files/cosine similarities/meta/cos_sim.parquet")
                for i in range(2,5):
                    mcs = pd.concat([mcs, pd.read_parquet(f"recommender_files/cosine similarities/meta/cos_sim{i}.parquet")])

                # obtaining app id of chosen game title
                game_id = mdts.loc[mdts["title"] == game_title].index

                # locating a similarity vector by app id
                cosine_similarity = mcs.loc[game_id, "cosine_similarity"].values[0]
                # combining similarity vector with app ids and sorting the list
                cosine_similarity = list(zip(cosine_similarity, mcs.index))
                cosine_similarity = sorted(cosine_similarity, reverse = True)

                # choosing app ids of top 25 games
                ids = [i[1] for i in cosine_similarity[1:26]]

                # locating games by app ids in the dataset
                table = mdts.loc[ids]

                platforms = ["Windows", "MacOS", "Linux", "SteamDeck"]

                # showing top 25 games from sorted dataset
                for index in table.index:

                    supported_platforms = list(mdts.loc[index, ["win", "mac", "linux", "steam_deck"]])
                    supported_platforms = list(zip(platforms, supported_platforms))
                    supported_platforms = [i[0] for i in supported_platforms if i[1] == True]

                    # creating an information card for each of the games
                    with st.container():

                        st.header(table.loc[index, "title"], divider = "rainbow")

                        col1, col2 = st.columns([1, 2])

                        with col1:
                            st.image(table.loc[index, "header_image"])

                        with col2:
                            if table.loc[index, 'rating'] == "Mixed":
                                st.subheader(f"_Rating_: :orange[{table.loc[index, 'rating']}]")
                            else:
                                st.subheader(f"_Rating_: :green[{table.loc[index, 'rating']}]")
                            st.text(f"Reliase date: {table.loc[index, 'date_release']}")
                            st.text(("Supported plarforms: " + ", ".join(supported_platforms)))
                            if table.loc[index, 'price_final'] == 0.0:
                                st.text(f"Price: Free")
                            else:
                                st.text(f"Price: {table.loc[index, 'price_final']}$")

top_25_similar_games_by_meta()

You can run the app by running the code in the next two cells.

***! This application will take ~30 seconds to load the results.***

In [ ]:
# restarting the Colab kernel to free all RAM
import os
os._exit(00)

In [ ]:
!pip install streamlit &> /dev/null
!npm install localtunnel &> /dev/null

!streamlit run recommender_files/cb_recommender_by_metadata.py & >/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

# And we are done
## Thanks for staying till the end